# Titanic - ML from disaster

Use machine learning to create a model that predicts which passengers survived the Titanic shipwreck.

In [50]:
import numpy as np
import pandas as pd

np.random.seed(13)

In [51]:
# Import training and test sets
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")

# Review dataset
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
test_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [53]:
# Drop columns deemed arbitrary
train = train_set.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test = test_set.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [54]:
# Review shape of training data
train.shape

(891, 9)

In [55]:
# Review shape of test data
test.shape

(418, 9)

In [56]:
# Set target variable and remove target from input data.

train_y = train[['Survived']]
test_y = test[['Survived']]

train_inputs = train.drop(['Survived'], axis=1)
test_inputs = test.drop(['Survived'], axis=1)

In [57]:
test_inputs.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

In [59]:
# Reveiw training dataset
train_inputs.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S


In [60]:
# Review column types
train_inputs.dtypes

PassengerId      int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
dtype: object

In [61]:
train_inputs['Pclass']=train_inputs['Pclass'].astype('object')

In [62]:
train_inputs.dtypes

PassengerId      int64
Pclass          object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
dtype: object

In [63]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [64]:
# Confirm numeric cols
numeric_columns

['PassengerId', 'Age', 'SibSp', 'Parch', 'Fare']

In [65]:
# Confirm cat cols
categorical_columns

['Pclass', 'Sex', 'Embarked']

In [66]:
# Create numeric pipeline
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [67]:
# Create cat pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [68]:
# Create preprocessor
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='drop')

In [69]:
# #Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x.shape

(891, 14)

In [70]:
#Fit and transform the test data
test_x = preprocessor.transform(test_inputs)

test_x.shape

(418, 14)

In [71]:
train_y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


In [72]:
# Determine a baseline accuracy using most frequent stra

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [73]:
# Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.6161616161616161


In [74]:
# Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.6363636363636364


In [75]:
# Create a single layer neural network model

import tensorflow as tf
from tensorflow import keras

np.random.seed(13)
tf.random.set_seed(13)

NN = keras.models.Sequential()

NN.add(keras.layers.Input(shape=(train_x.shape[1],)))
NN.add(keras.layers.Dense(16, activation='relu'))
NN.add(keras.layers.Dense(1, activation='sigmoid'))

In [76]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)

NN.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [77]:
# Fit the model
history = NN.fit(train_x, train_y, 
                    validation_data=(test_x, test_y), 
                    epochs=20, batch_size=100)

Epoch 1/20
9/9 [==============================] - 1s 18ms/step - loss: 0.8062 - accuracy: 0.4074 - val_loss: 0.7200 - val_accuracy: 0.5598
Epoch 2/20
9/9 [==============================] - 0s 6ms/step - loss: 0.5946 - accuracy: 0.7015 - val_loss: 0.5718 - val_accuracy: 0.7464
Epoch 3/20
9/9 [==============================] - 0s 7ms/step - loss: 0.5148 - accuracy: 0.7800 - val_loss: 0.4811 - val_accuracy: 0.7632
Epoch 4/20
9/9 [==============================] - 0s 6ms/step - loss: 0.4671 - accuracy: 0.8002 - val_loss: 0.4112 - val_accuracy: 0.7895
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4471 - accuracy: 0.8103 - val_loss: 0.3929 - val_accuracy: 0.8086
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4388 - accuracy: 0.8126 - val_loss: 0.3648 - val_accuracy: 0.8517
Epoch 7/20
9/9 [==============================] - 0s 5ms/step - loss: 0.4318 - accuracy: 0.8137 - val_loss: 0.3625 - val_accuracy: 0.8254
Epoch 8/20
9/9 [=================

In [78]:
train_scores = NN.evaluate(train_x, train_y, verbose=0)

print(f"Train {NN.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {NN.metrics_names[1]}: {train_scores[1]*100:.2f}%")

Train loss: 0.38
Train accuracy: 83.61%


In [79]:
test_scores = NN.evaluate(test_x, test_y, verbose=0)

print(f"Test {NN.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {NN.metrics_names[1]}: {test_scores[1]*100:.2f}%")

Test loss: 0.40
Test accuracy: 76.56%


In [80]:
# Create a multi-layer Deep Neural Network model

np.random.seed(13)
tf.random.set_seed(13)

DNN = keras.models.Sequential()

DNN.add(keras.layers.Input(train_x.shape[1],))
DNN.add(keras.layers.Dense(16, activation='relu'))
DNN.add(keras.layers.Dense(12, activation='relu'))
DNN.add(keras.layers.Dense(8, activation='relu'))
DNN.add(keras.layers.Dense(1, activation='sigmoid'))

In [81]:
# Compile model
adam = keras.optimizers.Adam(learning_rate=0.001)
DNN.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [82]:
# Apply early stopping to prevent overfitting

from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(patience=3, verbose=1)

callback = [earlystop]

DNN.fit(train_x, train_y, validation_data=(test_x, test_y), 
            epochs=100, batch_size=100, callbacks=callback)

Epoch 1/100
9/9 [==============================] - 1s 20ms/step - loss: 0.6921 - accuracy: 0.5937 - val_loss: 0.6512 - val_accuracy: 0.6555
Epoch 2/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6808 - accuracy: 0.6184 - val_loss: 0.6386 - val_accuracy: 0.6746
Epoch 3/100
9/9 [==============================] - 0s 5ms/step - loss: 0.6703 - accuracy: 0.6184 - val_loss: 0.6270 - val_accuracy: 0.6986
Epoch 4/100
9/9 [==============================] - 0s 6ms/step - loss: 0.6591 - accuracy: 0.6218 - val_loss: 0.6149 - val_accuracy: 0.7321
Epoch 5/100
9/9 [==============================] - 0s 6ms/step - loss: 0.6471 - accuracy: 0.6285 - val_loss: 0.6007 - val_accuracy: 0.7344
Epoch 6/100
9/9 [==============================] - 0s 6ms/step - loss: 0.6311 - accuracy: 0.6308 - val_loss: 0.5842 - val_accuracy: 0.7440
Epoch 7/100
9/9 [==============================] - 0s 6ms/step - loss: 0.6153 - accuracy: 0.6375 - val_loss: 0.5662 - val_accuracy: 0.7823
Epoch 8/100
9/9 [=========

In [83]:
train_scores = DNN.evaluate(train_x, train_y, verbose=0)

print(f"Train {DNN.metrics_names[0]}: {train_scores[0]:.2f}")

print(f"Train {DNN.metrics_names[1]}: {train_scores[1]*100:.2f}%")

Train loss: 0.42
Train accuracy: 81.82%


In [84]:
test_scores = DNN.evaluate(test_x, test_y, verbose=0)

print(f"Test {DNN.metrics_names[0]}: {test_scores[0]:.2f}")

print(f"Test {DNN.metrics_names[1]}: {test_scores[1]*100:.2f}%")

Test loss: 0.31
Test accuracy: 87.56%


In [85]:
# Now let's try a RandomForestClassifier model

from sklearn.ensemble import RandomForestClassifier 

rnd_clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, n_jobs=-1) 

rnd_clf.fit(train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=10, n_jobs=-1)

In [86]:
train_y_pred = rnd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9517396184062851


In [87]:
test_y_pred = rnd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.8181818181818182


# SVC Model 

In [88]:
from sklearn.svm import SVC

LSVC = SVC(kernel="linear")

LSVC.fit (train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [89]:
train_y_pred = LSVC.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.7867564534231201


In [90]:
test_y_pred = LSVC.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 1.0


## SVC Poly

In [91]:
from sklearn.svm import SVC
PSVC = SVC(kernel="poly", degree=3, coef0=1, C=.1)
PSVC.fit (train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.1, coef0=1, kernel='poly')

In [92]:
train_y_pred = PSVC.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.8338945005611672


In [93]:
test_y_pred = PSVC.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.9425837320574163


## SVC RBF

In [94]:
rbf_svm = SVC(kernel="rbf", C=2, gamma='scale')

rbf_svm.fit(train_x, train_y)

C:\Users\chadu\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=2)

In [95]:
train_y_pred = rbf_svm.predict(train_x)
train_acc = accuracy_score(train_y, train_y_pred)
print('Train acc: {}' .format(train_acc))

Train acc: 0.8462401795735129


In [96]:
test_y_pred = rbf_svm.predict(test_x)
test_acc = accuracy_score(test_y, test_y_pred)
print('Test acc: {}' .format(test_acc))

Test acc: 0.9138755980861244
